In [22]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
import math

# Methods

# Trapezoidal method

In [23]:
def trapezoidal_method(f : sp.Expr, a, b, n):
    # Spacing between points
    h = (b - a) / n
    # Set of points linearly spaced between a and b
    x = np.linspace(a, b, n+1)
    # Compute the value of the function at each point in x
    y = np.array(list(map(lambda var: f.subs('x', var), x)))
    # Compute the integral using the trapezoidal rule
    return h / 2 * (y[0] + 2 * np.sum(y[1:n]) + y[n])

def trapezoidal_method_error(f : sp.Expr, a, b, n):
    # Compute the second derivative of the function
    f_2 = sp.diff(f, 'x', 2)
    h = (b - a) / n
    random_value_on_interval = np.random.uniform(a, b)
    return -h**2 / 12 * f_2.subs('x', random_value_on_interval)

## Tests for trapezoidal method

In [24]:
a = 0
b = 1
n = 10
x = sp.symbols("x")
f = sp.exp(x)
integral = trapezoidal_method(f, a, b, n)
print(f"Integral of exp(x) from {a} to {b} using {n} points: {integral}")

Integral of exp(x) from 0 to 1 using 10 points: 1.71971349138931


In [25]:
error_expected = 10e-3
n=0
while True:
    n += 1
    error = trapezoidal_method_error(f, a, b, n)
    if abs(error) < error_expected:
        break
print(f"Number of points needed to get an error less than 10e-3: {n}, {1/(abs(error))}")

Number of points needed to get an error less than 10e-3: 4, 171.336630884473


# 1/3 Simpson Method

In [32]:
def simpson_one_third_method(f: sp.Expr, a, b, n) -> float:
    # Spacing between points
    h = (b - a) / n
    # Set of points linearly spaced between a and b
    x = np.linspace(a, b, n+1)
    # Compute the value of the function at each point in x
    y = np.array(list(map(lambda var: f.subs('x', var), x)))

    # Compute the integral using the Simpson's one third rule
    y_less_extremities = y[1:-1]
    y_even_positions = np.array(list(map(lambda val: val[1], filter(lambda val: val[0] % 2 == 0, enumerate(y_less_extremities)))))
    y_odd_positions = np.array(list(map(lambda val: val[1], filter(lambda val: val[0] % 2 != 0, enumerate(y_less_extremities)))))
    return h / 3 * (y[0] + 4 * np.sum(y_even_positions) + 2 * np.sum(y_odd_positions) + y[-1])

## Tests for 1/3 Simpson Method

In [33]:
a = 0
b = 1
n = 10
x = sp.symbols("x")
f = sp.exp(x)
integral = simpson_one_third_method(f, a, b, n)
print(f"Integral of exp(x) from {a} to {b} using {n} points: {integral}")

Integral of exp(x) from 0 to 1 using 10 points: 1.71828278192482


#  3/8 Simpson method

In [60]:
def simpson_three_eights_method(f:sp.Expr, a, b, n) -> float:
    # Spacing between points
    h = (b - a) / n
    # Set of points linearly spaced between a and b
    x = np.linspace(a, b, n+1)
    # Compute the value of the function at each point in x
    y = np.array(list(map(lambda var: f.subs('x', var), x)))

    # Compute the integral using the Simpson's three eight rule
    weights = np.array([1.0, 3.0, 3.0, 1.0])
    summation = sum(
        map(lambda sub_interval: np.dot(sub_interval, weights), 
            map(lambda index: y[index:index+4], range(0, len(y), 4))
          )
        )

    return h * (3 / 8) * summation

## Tests for 3/8 Simpson method

In [61]:
a = 0
b = 1
n = 7
x = sp.symbols("x")
f = sp.exp(x)
integral = simpson_three_eights_method(f, a, b, n)
print(f"Integral of exp(x) from {a} to {b} using {n} points: {integral}")

Integral of exp(x) from 0 to 1 using 7 points: 1.43575215966960
